In [1]:
import pandas as pd
import numpy as np
import requests
import time
from scrapy.selector import Selector
from scrapy.http import HtmlResponse
from selenium import webdriver
from bs4 import BeautifulSoup
import json

chromedriver = "/Users/edwardlee/Desktop/chromedriver"

In [2]:
browser = webdriver.Chrome(executable_path = chromedriver)

In [3]:
# Get all years for 1996-2016
browser.get('http://stats.nba.com/league/player/')
HTML = browser.page_source
year_data = Selector(text=HTML).xpath("//select[@name='Season']/option/text()").extract()
time.sleep(2)

In [4]:
clutch_para = ['Base', 'Advanced', 'Misc', 'Scoring', 'Usage']

In [57]:
clutch_base_cols = [u'PLAYER_ID', u'PLAYER_NAME',
       u'AGE', u'GP', u'W', u'L', u'MIN',
       u'FGM', u'FGA', u'FG_PCT', u'FG3M', u'FG3A', u'FG3_PCT', u'FTM', u'FTA', u'FT_PCT',
       u'OREB', u'DREB', u'REB', u'AST', u'TOV', u'STL', u'BLK',
       u'BLKA', u'PF', u'PFD', u'PTS', u'PLUS_MINUS', u'DD2', u'TD3', 'SEASON']

In [29]:
clutch_adv_cols = [u'PLAYER_ID', u'OFF_RATING', u'DEF_RATING', u'NET_RATING', u'AST_PCT', u'AST_TO',
       u'AST_RATIO', u'OREB_PCT', u'DREB_PCT', u'REB_PCT', u'TM_TOV_PCT',
       u'EFG_PCT', u'TS_PCT', u'USG_PCT', u'PACE', u'PIE', u'FGM_PG', 'SEASON']

In [30]:
clutch_misc_cols = [u'PLAYER_ID', u'PTS_OFF_TOV', u'PTS_2ND_CHANCE', u'PTS_FB', u'PTS_PAINT',
       u'OPP_PTS_OFF_TOV', u'OPP_PTS_2ND_CHANCE', u'OPP_PTS_FB',
       u'OPP_PTS_PAINT', 'SEASON']

In [31]:
clutch_scoring_cols = [u'PLAYER_ID', u'PCT_FGA_2PT', u'PCT_FGA_3PT', u'PCT_PTS_2PT', u'PCT_PTS_2PT_MR',
       u'PCT_PTS_3PT', u'PCT_PTS_FB', u'PCT_PTS_FT', u'PCT_PTS_OFF_TOV',
       u'PCT_PTS_PAINT', u'PCT_AST_2PM', u'PCT_UAST_2PM', u'PCT_AST_3PM',
       u'PCT_UAST_3PM', u'PCT_AST_FGM', u'PCT_UAST_FGM', 'SEASON']

In [32]:
clutch_usage_cols = [u'PLAYER_ID', u'PCT_FGM', u'PCT_FGA', u'PCT_FG3M', u'PCT_FG3A',
       u'PCT_FTM', u'PCT_FTA', u'PCT_OREB', u'PCT_DREB', u'PCT_REB',
       u'PCT_AST', u'PCT_TOV', u'PCT_STL', u'PCT_BLK', u'PCT_BLKA', u'PCT_PF',
       u'PCT_PFD', u'PCT_PTS', 'SEASON']

In [58]:
### Merge per season/year by all stats/columns in that season
for year in year_data:
    base_csv = pd.read_csv('/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/rank_project/clutch_stats/Base_%s_per48.csv' % (year))
    adv_csv = pd.read_csv('/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/rank_project/clutch_stats/Advanced_%s_per48.csv' % (year))
    misc_csv = pd.read_csv('/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/rank_project/clutch_stats/Misc_%s_per48.csv' % (year))
    score_csv = pd.read_csv('/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/rank_project/clutch_stats/Scoring_%s_per48.csv' % (year))
    usage_csv = pd.read_csv('/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/rank_project/clutch_stats/Usage_%s_per48.csv' % (year))
    
    base_csv['SEASON'] = [year for i in range(base_csv.shape[0])]
    adv_csv['SEASON'] = [year for i in range(adv_csv.shape[0])]
    misc_csv['SEASON'] = [year for i in range(misc_csv.shape[0])]
    score_csv['SEASON'] = [year for i in range(score_csv.shape[0])]
    usage_csv['SEASON'] = [year for i in range(usage_csv.shape[0])]
    
    base_csv  = base_csv[clutch_base_cols]
    adv_csv   = adv_csv[clutch_adv_cols]
    misc_csv  = misc_csv[clutch_misc_cols]
    score_csv = score_csv[clutch_scoring_cols]
    usage_csv = usage_csv[clutch_usage_cols]
    
    df_merge = base_csv.merge(adv_csv, on=['PLAYER_ID', 'SEASON'], how='outer') \
                       .merge(misc_csv, on=['PLAYER_ID', 'SEASON'], how='outer') \
                       .merge(score_csv, on=['PLAYER_ID', 'SEASON'], how='outer') \
                       .merge(usage_csv, on=['PLAYER_ID', 'SEASON'], how='outer')
    
    link = '/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/rank_project/clutch_stats/clutch_merged_%s.csv' % year
    df_merge.to_csv(link)

In [59]:
### Save all columns that I want to aggregate the ranks together, in case 'Unnamed: 0' column appears
clutch_all_cols = [u'GP', u'W', u'L', u'MIN',
       u'FGM', u'FGMiss', u'FG_PCT', u'FG3M', u'FG3Miss', u'FG3_PCT', u'FTM', u'FTMiss', u'FT_PCT',
       u'OREB', u'DREB', u'REB', u'AST', u'TOV', u'STL', u'BLK',
       u'BLKA', u'PF', u'PFD', u'PTS', u'PLUS_MINUS', u'DD2', u'TD3',
       u'OFF_RATING', u'DEF_RATING', u'NET_RATING', u'AST_PCT', u'AST_TO',
       u'AST_RATIO', u'OREB_PCT', u'DREB_PCT', u'REB_PCT', u'TM_TOV_PCT',
       u'EFG_PCT', u'TS_PCT', u'USG_PCT', u'PACE', u'PIE', u'FGM_PG',
       u'PTS_OFF_TOV', u'PTS_2ND_CHANCE', u'PTS_FB', u'PTS_PAINT',
       u'OPP_PTS_OFF_TOV', u'OPP_PTS_2ND_CHANCE', u'OPP_PTS_FB',
       u'OPP_PTS_PAINT',u'PCT_FGA_2PT', u'PCT_FGA_3PT', u'PCT_PTS_2PT', u'PCT_PTS_2PT_MR',
       u'PCT_PTS_3PT', u'PCT_PTS_FB', u'PCT_PTS_FT', u'PCT_PTS_OFF_TOV',
       u'PCT_PTS_PAINT', u'PCT_AST_2PM', u'PCT_UAST_2PM', u'PCT_AST_3PM',
       u'PCT_UAST_3PM', u'PCT_AST_FGM', u'PCT_UAST_FGM', u'PCT_FGM', u'PCT_FGA', u'PCT_FG3M', u'PCT_FG3A',
       u'PCT_FTM', u'PCT_FTA', u'PCT_OREB', u'PCT_DREB', u'PCT_REB',
       u'PCT_AST', u'PCT_TOV', u'PCT_STL', u'PCT_BLK', u'PCT_BLKA', u'PCT_PF',
       u'PCT_PFD', u'PCT_PTS']

In [76]:
### Sort values by every column and give a rank and aggregate rank per player per season
for year in year_data:
    csv = '/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/rank_project/clutch_stats/clutch_merged_%s.csv' % year
    df_tmp = pd.read_csv(csv)
    
    # Added Columns Misses and use Misses instead of Attempts
    df_tmp['FGMiss'] = df_tmp['FGA'] - df_tmp['FGM']
    df_tmp['FTMiss'] = df_tmp['FTA'] - df_tmp['FTM']
    df_tmp['FG3Miss'] = df_tmp['FG3A'] - df_tmp['FG3M']
    
    # Limit only to players with more than 10 games played and more than 20 total minutes
    df_tmp = df_tmp[(df_tmp['GP'] > 10) & (df_tmp['MIN'] > 20.)]
    
    # Rank by sort values of every column and creating a new column
    for col in clutch_all_cols:
        if (col == 'L') or (col == 'TOV') or (col == 'PF') or (col == 'PCT_TOV') or (col == 'PCT_PF'):
            df_tmp.sort_values(col, ascending=True, inplace=True)
            new_col_name = 'rank_%s' % col
            df_tmp[new_col_name] = [i+1 for i in range(df_tmp.shape[0])]
        elif (col == 'OPP_PTS_OFF_TOV') or (col == 'OPP_PTS_2ND_CHANCE') or (col == 'OPP_PTS_FB') or (col == 'OPP_PTS_PAINT'):
            df_tmp.sort_values(col, ascending=True, inplace=True)
            new_col_name = 'rank_%s' % col
            df_tmp[new_col_name] = [i+1 for i in range(df_tmp.shape[0])]
        else:
            df_tmp.sort_values(col, ascending=False, inplace=True)
            new_col_name = 'rank_%s' % col
            df_tmp[new_col_name] = [i+1 for i in range(df_tmp.shape[0])]
    
    # Save all rank columns into a list
    ranked_cols = ['PLAYER_ID', 'PLAYER_NAME', 'SEASON']
    for col in df_tmp.columns:
        if 'rank' in col:
            ranked_cols.append(col)
    
    # Aggregate every players rank
    df_tmp = df_tmp[ranked_cols]
    df_tmp['agg_rank'] = df_tmp.ix[:,2:].mean(axis=1)
    
    link = '/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/rank_project/clutch_stats/clutch_merged_%s_withRANK.csv' % year
    df_tmp.to_csv(link)

In [83]:
### MERGE CLUTCH STATS BY CATEGORIES FROM 1996-97 to 2015-16

clutch_rank = []

for year in year_data:
    csv_file = '/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/rank_project/clutch_stats/clutch_merged_%s_withRANK.csv' % year
    df = pd.read_csv(csv_file)
    df.drop('Unnamed: 0', axis=1, inplace=True)

    clutch_rank.append(df)

clutch_rank_df = pd.concat(clutch_rank)
clutch_rank_df.to_csv('/Users/edwardlee/Desktop/PROJECTS/NBA Fantasy Project/rank_project/completed/clutch_rank_target.csv')